# Trees
#### Basic binary tree
Implement a nodal tree structure and methods to identify:
- if a node is a leaf
- the values of a node's children
- the values of its grandchildren
- the size of it's subree (the node and all of its descendants)
- the height of its subtree.

In [2]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def is_leaf(self):
        return not self.left and not self.right

    def children(self):
        c = []
        if self.left:
            c.append(self.left.val)
        if self.right:
            c.append(self.right.val)
        return c

    def grandchildren(self):
        g = []
        b = ['left', 'right']
        for child in [self.left, self.right]:
            if child and child.left:
                g.append(child.left.val)
            if child and child.right:
                g.append(child.right.val)
        return g

    def subtree_size(self):
        def component_size(node):
            if node:
                return 1 + component_size(node.left) + component_size(node.right)
            else:
                return 0
        return component_size(self)
    
    def subtree_height(self):
        levels = set()
        def child_check(current_node, l):
            if current_node:
                levels.add(l)
                child_check(current_node.left, l+1)
                child_check(current_node.right, l+1)
            return
        child_check(self,1)
        return max(levels)

In [3]:
my_node = Node(5)
print(f"First node value: {my_node.val}")
print(f"Is first node a leaf? {my_node.is_leaf()}")
print("Adding more nodes to tree...")
my_node.left = Node('l')
my_node.left.right = Node('R')
my_node.right = Node('r')
my_node.right.right = Node('R')
print(f"Is first node still a leaf? {my_node.is_leaf()}")
print(f"What are the first node's children's values? {my_node.children()}")
print(f"And the grandchildren's values? {my_node.grandchildren()}")
print(f"The size of the subtree is now {my_node.subtree_size()} and the height is {my_node.subtree_height()}")
print(f"And the size of the right child's subtree is {my_node.right.subtree_size()} and the height is {my_node.right.subtree_height()}")


First node value: 5
Is first node a leaf? True
Adding more nodes to tree...
Is first node still a leaf? False
What are the first node's children's values? ['l', 'r']
And the grandchildren's values? ['R', 'R']
The size of the subtree is now 5 and the height is 3
And the size of the right child's subtree is 2 and the height is 2


What if we need to find the parent node? The following implementation returns...
- whether a node is the root
- the IDs of all ancestors
- the depth of a node
- the lowest common ancestor (LCA) of two nodes
- the distance, or number of edges in the path, bettween two nodes

In [5]:
class Node:
    def __init__(self, id, parent=None, left=None, right=None):
        self.id = id
        self.parent = parent
        self.left = left
        self.right = right

    def is_root(self):
        return not self.parent

    def ancestors(self):
        a = []
        curr = self
        while curr.parent:
            a.append(curr.parent.id)
            curr = curr.parent
        return a

    def depth(self):
        d = 0
        curr = self
        while curr.parent:
            d += 1
            curr = curr.parent
        return d

    def lca(self, node2):
        d = self.depth()
        d2 = node2.depth()
        curr, curr2 = self, node2
        while d2 > d:
            curr2 = curr2.parent
            d2 -= 1
        while d > d2:
            curr = curr.parent
            d -= 1
        while curr != curr2:
            if not curr.parent:
                raise Exception("No common ancestor")
            curr, curr2 = curr.parent, curr2.parent
        return curr.id

    def distance(self, node2):
        d1, d2 = self.depth(), node2.depth()
        d = d1 + d2
        curr, curr2 = self, node2
        while d2 > d1:
            curr2 = curr2.parent
            d2 -= 1
        while d1 > d2:
            curr = curr.parent
            d1 -= 1
        while curr != curr2:
            if not curr.parent:
                raise Exception("Nodes not connected")
            d1 -= 1
            d2 -= 1
            curr, curr2 = curr.parent, curr2.parent
        return d - d1 - d2       

In [6]:
my_node = Node('a')
print(f"First node id: {my_node.id}")
print(f"Is first node the root? {my_node.is_root()}")
print("Adding more nodes to tree...")
node_b = Node('b',parent=my_node)
my_node.left = node_b
my_node.left.left = Node('d', parent=my_node.left)
node_e = Node('e', parent=node_b)
node_b.right = node_e
my_node.right = Node('c', parent=my_node)
node_f = Node('f', parent=my_node.right)
my_node.right.left = node_f
node_h = Node('h', parent=node_b.left)
node_b.left.left = node_h
my_node.left.left.right = Node('i', parent=node_b.left)
my_node.right.left.left = Node('j', parent=node_f)
node_j = my_node.right.left.left
my_node.right.left.right = Node('k', parent=node_f)
print(f"Is first node still the root? {my_node.is_root()}")
print(f"Is node_j a root? {node_j.is_root()}")
print(f"What are node_j's ancestors? {node_j.ancestors()}")
print(f"And node_h's ancestors? {node_h.ancestors()}")
print(f"What are the depths of nodes a, b, f and h? a:{my_node.depth()}, b:{node_b.depth()}, f:{node_f.depth()}, h:{node_h.depth()}")
print(f"LCA between h and e is {node_h.lca(node_e)}; distance is {node_h.distance(node_e)}")
print(f"LCA between j and f is {node_j.lca(node_f)}; distance is {node_j.distance(node_f)}")

First node id: a
Is first node the root? True
Adding more nodes to tree...
Is first node still the root? True
Is node_j a root? False
What are node_j's ancestors? ['f', 'c', 'a']
And node_h's ancestors? ['d', 'b', 'a']
What are the depths of nodes a, b, f and h? a:0, b:1, f:2, h:3
LCA between h and e is b; distance is 3
LCA between j and f is f; distance is 1


#### Aligned chain
Given a binary tree, return the length of the longest descendant chain of nodes whose ids align with their depth.

In [8]:
# Example binary tree
new = Node(7)
new.left = Node(1, parent=new)
new.left.left = Node(2, parent=new.left)
new.left.left.left = Node(4, parent=new.left.left)
new.left.left.right = Node(3, parent=new.left.left)
new.left.right = Node(8, parent=new.left)
new.right = Node(3, parent=new)
new.right.left = Node(2, parent=new.right)
new.right.left.left = Node(3, parent=new.right.left)
new.right.left.right = Node(3, parent=new.right.left)

In [12]:
# Solution uses a recrsive function that makes a preorder traversal of the tree from the root
def aligned_chain(node, chain_length=0, max_chain=0, depth=0):
    
    if not node:
        #print(f'node not found, returning max_chain ', max_chain)
        return max_chain
    #print(f'running function on node {node.id} with parameters cl={chain_length}, mc={max_chain}, d={depth}')
    if node.id == depth:
        #print(f'node id = depth = {depth}. We have a winner!')
        chain_length +=1
        #print(f'chain incremented to ', chain_length)
        max_chain = max(max_chain, chain_length)
        #print(f'max_chain now ', max_chain)
    else:
        chain_length = 0
        #print(f'node id != depth. Chain length reset to 0')
    depth += 1
    #print(f'depth incremented to ', depth)
    #print(f'will now run function on left and right children of node {node.id}')
    return max(aligned_chain(node.left, chain_length, max_chain, depth), aligned_chain(node.right, chain_length, max_chain, depth))

In [14]:
aligned_chain(new)

3